In [1]:
%load_ext autoreload
%autoreload 2
#os.environ["OMP_NUM_THREADS"] = str(os.cpu_count() // 2)
#os.environ["GOMP_CPU_AFFINITY"] = "granularity=core,compact"
from Scripts.model import *
from Scripts.loss import *
from Scripts.results_manager import *
from Scripts.plots import *
from Scripts.dataset import *
from Scripts.trainer import *
from Scripts.inference import *
from Scripts.Onnx_Class import *
from Scripts.lr_finder import *
from Scripts.generate_configs import *
#from Scripts.asymmetric_model import *
from Scripts.generate_asymmetric_configs import *
from Scripts.excecute import *
from Scripts.quantize import *
from Scripts.upload_summaries import *

c:\Users\alant\AppData\Local\Programs\Python\Python313\Lib\site-packages\openvino\runtime\__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(


In [ ]:
config_path = "Configs"

In [ ]:
training_selected_class(config_path, 'grid')
metrics_to_db()

In [ ]:
training_all_classes(config_path)
metrics_to_db()

In [ ]:
training_run_folder = "Training_Runs"
inference_output_dir = "Inference_Runs"

inference_model(training_run_folder, inference_output_dir)

In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image

# 1. ONNX-Modell laden
onnx_model_path = r"onnx_models\STFPM_mobilenetv4_conv_large.onnx"
sess = ort.InferenceSession(onnx_model_path)

image_path = glob.glob(os.path.join(
    r'Images\grid\test\broken', '*.png'))
for path in image_path:

    # 2. Ein Bild für die Inferenz vorbereiten
    # image_path = r"Images\grid\test\bent\003.png"
    image = Image.open(path).convert("RGB")

    # Resize the image to the expected input size
    img_size = 256
    image = image.resize((img_size, img_size))

    input_data = np.array(image, dtype=np.uint8)
    input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension

    # 3. Inferenz durchführen
    input_name = sess.get_inputs()[0].name
    outputs = sess.run(None, {input_name: input_data})

    # 4. Ergebnisse auswerten
    anomaly_map = outputs[0]
    anomaly_score = outputs[1]

    print(f"Anomalie-Score für das Bild: {anomaly_score[0]}")

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # 2. Originalbild anzeigen
    axes[0].imshow(image)
    axes[0].set_title("Originalbild")
    axes[0].axis('off')

    # 3. Heatmap über das Originalbild legen
    axes[1].imshow(image)
    heatmap = axes[1].imshow(np.squeeze(anomaly_map), cmap='jet', alpha=0.5)
    axes[1].set_title("Anomalie-Heatmap")
    axes[1].axis('off')

    # 4. Farbbalken für die Heatmap hinzufügen
    fig.colorbar(heatmap, ax=axes[1], fraction=0.046, pad=0.04)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    # 5. Plot anzeigen oder speichern
    plt.suptitle("Vergleich: Originalbild vs. Anomalie-Heatmap")
    plt.show()

In [2]:
config = load_config(
    r'Training_Runs\MVTecAD_grid\resnet18\2830be19-bea8-41f8-a1e8-1a6ef49b8cbc\STFPM_Config_resnet18.yaml')
with open(r'Training_Runs\MVTecAD_grid\resnet18\2830be19-bea8-41f8-a1e8-1a6ef49b8cbc\summary_metrics.json', 'r') as f:
    summary_metrics = json.load(f)
quantize_model(
    r'Training_Runs\MVTecAD_grid\resnet18\2830be19-bea8-41f8-a1e8-1a6ef49b8cbc\weights\resnet18_grid_2830be19-bea8-41f8-a1e8-1a6ef49b8cbc_best_student.pth',
    config,
    summary_metrics
)

Quantization finished — saved to disk.


In [3]:
test_set = MVTecDataset(
    img_size=config['dataset']['img_size'],
    base_path=config['dataset']['base_path'],
    cls=config['dataset']['class'],
    mode='test',
    download_if_missing=False
)
test_loader = DataLoader(
    test_set,
    batch_size=config['dataloader']['batch_size'],
    shuffle=False
)

In [4]:
torch.backends.quantized.engine = 'fbgemm'

In [5]:
cpu_device = torch.device("cpu")

base_model_for_loading = STFPM(
    architecture=config['model']['architecture'],
    layers=config['model']['layers']
).to(cpu_device).eval()

model_to_load = QuantizableWrapper(
    base_model_for_loading).to(cpu_device).eval()

fuse_resnet_student_model(model_to_load)

model_to_load.student_model.qconfig = torch.quantization.get_default_qconfig(
    'fbgemm')

torch.quantization.prepare(model_to_load.student_model, inplace=True)
torch.quantization.convert(model_to_load.student_model, inplace=True)

model_to_load.student_model.load_state_dict(torch.load(
    r"quantized_models\resnet18\2830be19-bea8-41f8-a1e8-1a6ef49b8cbc\quantized_model.pth",
    map_location=cpu_device
))
model_to_load.eval()

print("Quantized model loaded successfully for inference.")

config['device']['type'] = 'cpu'

infer = Inference(
    model=model_to_load,
    test_loader=test_loader,
    config=config,
    path_to_student_weight=None,
    trainings_id=summary_metrics.get('training_id'),
    inferenz=True
)

auroc, inf_time = infer.evaluate_per_epoch()
print(f"AUROC: {auroc}, Inference Time: {inf_time}")

C:\Users\alant\AppData\Local\Temp\ipykernel_3516\831047794.py:16: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  torch.quantization.prepare(model_to_load.student_model, inplace=True)
c:\Users\alant\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\ao\quantization\observer.py:246: UserWarning: Please use quant_min and quant_max to specify the range for observ

Quantized model loaded successfully for inference.


NotImplementedError: Could not run 'quantized::conv2d_relu.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d_relu.new' is only available for these backends: [Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMTIA, AutogradMAIA, AutogradMeta, Tracer, AutocastCPU, AutocastMTIA, AutocastMAIA, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDisp

In [ ]:
print(model_to_load.student_model)